## Data Processing

In [93]:
import pandas as pd

In [94]:
appointments_df = pd.read_csv("../../data/appointments.csv")
office_hours_df = pd.read_csv("../../data/office_hours.csv")
office_locations_df = pd.read_csv("../../data/office_locations.csv")
services_df = pd.read_csv("../../data/services.csv")

In [95]:
appointments_df.head()

,appointment_id,customer_id,service_id,office_id,appointment_date,appointment_time,status
0,APT001,CUST101,PS001,LOC001,2024-12-15,09:30,Confirmed
1,APT002,CUST102,PS003,LOC002,2024-12-15,14:00,Confirmed
2,APT003,CUST103,PS006,LOC003,2024-12-16,11:30,Pending
3,APT004,CUST104,PS008,LOC002,2024-12-16,15:30,Confirmed
4,APT005,CUST105,PS004,LOC004,2024-12-17,10:00,Cancelled


In [96]:
office_hours_df.head()

,office_id,day_of_week,opening_time,closing_time
0,LOC001,Monday,08:00,17:00
1,LOC001,Tuesday,08:00,17:00
2,LOC001,Wednesday,08:00,17:00
3,LOC001,Thursday,08:00,17:00
4,LOC001,Friday,08:00,17:00


In [97]:
office_locations_df.head()

,office_id,location_name,address,region,phone_number
0,LOC001,Paysoko CBD,Kimathi Street Business Center 3rd Floor,Central Nairobi,+254-20-5550101
1,LOC002,Paysoko Westlands,The Mall 2nd Floor Westlands Road,Westlands,+254-20-5550102
2,LOC003,Paysoko Eastleigh,Eastleigh Mall Ground Floor 1st Avenue,Eastleigh,+254-20-5550103
3,LOC004,Paysoko Karen,Karen Shopping Center Block B,Karen,+254-20-5550104
4,LOC005,Paysoko Kasarani,Lucky City Mall Ground Floor,Kasarani,+254-20-5550105


In [98]:
services_df.head()

,service_id,service_name,description,cost_ksh,duration_minutes
0,PS001,Money Transfer,Domestic money transfer between bank accounts ...,150,15
1,PS002,Bill Payment,Payment of utility bills including electricity...,100,10
2,PS003,International Remittance,International money transfer services to vario...,500,20
3,PS004,Business Payments,B2B payment solutions and bulk payment processing,300,25
4,PS005,Mobile Top-up,Airtime and data bundle purchase for all major...,50,5


## Connect To Database

In [99]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os
from typing import Optional
from neo4j.exceptions import ServiceUnavailable, AuthError

def connect_to_neo4j() -> Optional[GraphDatabase.driver]:
    """
    Establishes a connection to Neo4j database using environment variables.
    
    Required environment variables:
    - NEO4J_URI: The URI of the Neo4j database
    - NEO4J_USERNAME: Username for authentication
    - NEO4J_PASSWORD: Password for authentication
    
    Returns:
        Neo4j driver instance if connection is successful, None otherwise
    """
    # Load environment variables from .env file
    load_dotenv()
    
    # Get database credentials from environment variables
    uri = os.getenv("NEO4J_URI")
    username = os.getenv("NEO4J_USERNAME")
    password = os.getenv("NEO4J_PASSWORD")
    
    # Verify all required environment variables are present
    if not all([uri, username, password]):
        missing_vars = [var for var, val in 
                       [("NEO4J_URI", uri), 
                        ("NEO4J_USERNAME", username), 
                        ("NEO4J_PASSWORD", password)] 
                       if not val]
        print(f"Error: Missing required environment variables: {', '.join(missing_vars)}")
        return None
    
    try:
        # Create driver instance
        driver = GraphDatabase.driver(uri, auth=(username, password))
        
        # Verify connection
        driver.verify_connectivity()
        print("Successfully connected to Neo4j database")
        return driver
    
    except AuthError as e:
        print(f"Authentication error: {str(e)}")
        return None
    except ServiceUnavailable as e:
        print(f"Neo4j database is not available: {str(e)}")
        return None
    except Exception as e:
        print(f"An error occurred while connecting to Neo4j: {str(e)}")
        return None

def close_neo4j_connection(driver: GraphDatabase.driver) -> None:
    """
    Safely closes the Neo4j database connection.
    
    Args:
        driver: Neo4j driver instance to close
    """
    if driver:
        driver.close()
        print("Neo4j connection closed successfully")


In [100]:
driver = connect_to_neo4j()

Successfully connected to Neo4j database


## Insert Into Neo4j Functions

In [101]:
from neo4j import GraphDatabase
from typing import List, Dict

### Load Office Locations Data Into Neo4j

In [102]:
def insert_offices_df(df: pd.DataFrame, driver: GraphDatabase.driver) -> bool:
    """Takes data from the office_locations.csv file and inserts into Neo4j database.
    Args:
        driver: Neo4j driver instance
        df: Pandas dataframe of offices CSV file
    """
    try:
        # Create constraints if they don't exist (run only once)
        create_constraints(driver)
            
        # Get offices
        offices = df.to_dict("records")
        
        with driver.session() as session:
            for office in offices:
                result = session.execute_write(create_office_location, office)
                print(f"Created office: {result}")
                
        return True
    
    except Exception as e:
        print(f"Error importing office locations: {str(e)}")
        return False
        

    
def create_office_location(tx, office: Dict) -> str:
    """
    Creates an office location node in Neo4j.
    
    Args:
        tx: Neo4j transaction
        office: Dictionary containing office location data
    
    Returns:
        String confirming creation of office location
    """
    # Cypher query to create office location
    query = """
    MERGE (o:OfficeLocation {office_id: $office_id})
    SET 
        o.location_name = $location_name,
        o.address = $address,
        o.region = $region,
        o.phone_number = $phone_number,
        o.created_at = datetime(),
        o.last_updated = datetime()
    RETURN o.location_name + ' created/updated successfully' as result
    """
    
    # Execute query with parameters
    result = tx.run(query, 
                   office_id=office['office_id'],
                   location_name=office['location_name'],
                   address=office['address'],
                   region=office['region'],
                   phone_number=office['phone_number'])
    
    return result.single()['result']


def create_constraints(driver: GraphDatabase.driver) -> None:
    """Creates unique constraints for office locations, based on office_id"""
    with driver.session() as session:
        # Create constraint on office_id if it doesn't exist
        try:
            session.run("""
                CREATE CONSTRAINT office_id_unique IF NOT EXISTS
                FOR (o:OfficeLocation) REQUIRE o.office_id IS UNIQUE
            """)
        except Exception as e:
            print(f"Error creating constraint: {str(e)}")

In [103]:

insert_offices_df(office_locations_df, driver)

Created office: Paysoko CBD created/updated successfully
Created office: Paysoko Westlands created/updated successfully
Created office: Paysoko Eastleigh created/updated successfully
Created office: Paysoko Karen created/updated successfully
Created office: Paysoko Kasarani created/updated successfully


True

### Load Operating Hours Dataset Into Neo4j

In [104]:
def insert_offices_housr_df(df: pd.DataFrame, driver: GraphDatabase.driver) -> bool:
    """Takes data from the office_hours.csv file and inserts into Neo4j database.
    Args:
        driver: Neo4j driver instance
        df: Pandas dataframe of offices CSV file
    """
    try:
            
        # Get offices
        offices_hour = df.to_dict("records")
        
        with driver.session() as session:
            for office_hour in offices_hour:
                result = session.execute_write(create_office_hour, office_hour)
                print(f"Created office hours: {result}")
                
        return True
    
    except Exception as e:
        print(f"Error importing office hour: {str(e)}")
        return False
        
        
def create_office_hour(txt, office_hour_data: Dict) -> str:
    """Insert a given office hour data into Neo4j database
    
    Args:
        tx: Neo4j transaction
        office_hour_data: Dictionary containing office location data
    
    Returns:
        String confirming creation of office location
    """
    
    try:
        query = """
        MERGE (oh:OfficeHour {office_id: $office_id, day_of_week: $day_of_week})
        SET
            oh.opening_time = $opening_time,
            oh.closing_time = $closing_time,
            oh.created_at = datetime(),
            oh.last_updated = datetime()
        WITH oh
        MATCH (o:OfficeLocation {office_id: $office_id})
        CREATE (o)-[:WORKING_HOURS]->(oh)
        RETURN $office_id + ' created/updated successfully' as result
        """
        
        result = txt.run(
            query,
            office_id=office_hour_data['office_id'],
            day_of_week=office_hour_data['day_of_week'],
            opening_time = office_hour_data["opening_time"],
            closing_time = office_hour_data["closing_time"],
        )
        
        return result.single()["result"]
    
    except Exception as e:
        print(f"Error occured inserting office hour: {e}")

In [105]:
insert_offices_housr_df(office_hours_df, driver)

Created office hours: LOC001 created/updated successfully
Created office hours: LOC001 created/updated successfully
Created office hours: LOC001 created/updated successfully
Created office hours: LOC001 created/updated successfully
Created office hours: LOC001 created/updated successfully
Created office hours: LOC001 created/updated successfully
Created office hours: LOC001 created/updated successfully
Created office hours: LOC002 created/updated successfully
Created office hours: LOC002 created/updated successfully
Created office hours: LOC002 created/updated successfully
Created office hours: LOC002 created/updated successfully
Created office hours: LOC002 created/updated successfully
Created office hours: LOC002 created/updated successfully
Created office hours: LOC002 created/updated successfully
Created office hours: LOC003 created/updated successfully
Created office hours: LOC003 created/updated successfully
Created office hours: LOC003 created/updated successfully
Created office

True

### Load Services Data Into Neo4j

In [106]:
def insert_service_df(df: pd.DataFrame, driver: GraphDatabase.driver) -> bool:
    """Takes data from the services.csv file and inserts into Neo4j database.
    Args:
        driver: Neo4j driver instance
        df: Pandas dataframe of offices CSV file
    """
    try:
            
        # Get offices
        services = df.to_dict("records")
        
        with driver.session() as session:
            for service in services:
                result = session.execute_write(create_service, service)
                print(f"Created service: {result}")
                
        return True
    
    except Exception as e:
        print(f"Error importing service data: {str(e)}")
        return False
        
        
def create_service(txt, service_data: Dict) -> str:
    """Insert a given service data into Neo4j database
    
    Args:
        tx: Neo4j transaction
        service_data: Dictionary containing service data
    
    Returns:
        String confirming creation of service name
    """
    
    try:
        query = """
        MERGE (s:Services {service_id: $service_id, service_name: $service_name})
        SET
            s.description = $description,
            s.cost_ksh = $cost_ksh,
            s.duration_minutes = $duration_minutes,
            s.created_at = datetime(),
            s.last_updated = datetime()
        RETURN $service_name + ' created/updated successfully' as result
        """
        
        result = txt.run(
            query,
            service_id=service_data['service_id'],
            service_name=service_data['service_name'],
            description = service_data["description"],
            cost_ksh = service_data["cost_ksh"],
            duration_minutes = service_data["duration_minutes"],
        )
        
        return result.single()["result"]
    
    except Exception as e:
        print(f"Error occurred inserting new service: {e}")

In [107]:
insert_service_df(services_df, driver)

Created service: Money Transfer created/updated successfully
Created service: Bill Payment created/updated successfully
Created service: International Remittance created/updated successfully
Created service: Business Payments created/updated successfully
Created service: Mobile Top-up created/updated successfully
Created service: Government Payments created/updated successfully
Created service: School Fee Payment created/updated successfully
Created service: Merchant Services created/updated successfully


True

### Load Appointments Data Into Neo4j

In [108]:
def insert_appointment_df(df: pd.DataFrame, driver: GraphDatabase.driver) -> bool:
    """Takes data from the appointments.csv file and inserts into Neo4j database.
    Args:
        driver: Neo4j driver instance
        df: Pandas dataframe of offices CSV file
    """
    try:
        
        # Create appointment id constraint
        create_appointment_constraints(driver=driver)
            
        # Get offices
        appointments = df.to_dict("records")
        
        with driver.session() as session:
            for appointment in appointments:
                result = session.execute_write(create_appointment, appointment)
                print(f"Scheduled appointment: {result}")
                
        return True
    
    except Exception as e:
        print(f"Error importing appointment data: {str(e)}")
        return False
        
        
def create_appointment(txt, appointment_data: Dict) -> str:
    """Insert a given appointments data into Neo4j database
    
    Args:
        tx: Neo4j transaction
        appointment_details: Dictionary containing appointment details
    
    Returns:
        String confirming creation of appointment id
    """
    
    try:
        # NOTE: What goes into MERGE must be unique eg: appointment_id
        query = """
        MERGE (a:Appointment {appointment_id: $appointment_id})
        SET
            a.customer_id = $customer_id,
            a.office_id = $office_id,
            a.service_id = $service_id,
            a.appointment_date = $appointment_date,
            a.appointment_time = $appointment_time,
            a.status = $status,
            a.created_at = datetime(),
            a.last_updated = datetime()
        WITH a
        MATCH (o:OfficeLocation {office_id: $office_id})
        MATCH (s:Services {service_id: $service_id})
        MERGE (a)-[:SCHEDULED_AT {
            status: $status,
            created_at: datetime()
        }]->(o)
        MERGE (a)-[:FOR_SERVICE {
            status: $status,
            created_at: datetime()
        }]->(s)
        RETURN $appointment_id + ' created/updated successfully' as result
        """
        
        result = txt.run(
            query,
            appointment_id=appointment_data['appointment_id'],
            customer_id=appointment_data['customer_id'],
            office_id = appointment_data["office_id"],
            appointment_date = appointment_data["appointment_date"],
            appointment_time = appointment_data["appointment_time"],
            status = appointment_data["status"],
            service_id = appointment_data["service_id"],
        )
        
                # Properly handle the result
        record = result.single()
        if record:
            return record['result']
        
        return f"Appointment {appointment_data['appointment_id']} created but no result returned"
    
    
    except Exception as e:
        print(f"Error occurred inserting new appointment: {e}")
        
        
def create_appointment_constraints(driver: GraphDatabase.driver) -> None:
    """Creates unique constraints for appointment id field"""
    with driver.session() as session:
        # Create constraint on office_id if it doesn't exist
        try:
            session.run("""
                CREATE CONSTRAINT appointment_id_unique IF NOT EXISTS
                FOR (a:Appointment) REQUIRE a.appointment_id IS UNIQUE
            """)
        except Exception as e:
            print(f"Error creating constraint: {str(e)}")

In [109]:
insert_appointment_df(appointments_df, driver)

Scheduled appointment: APT001 created/updated successfully
Scheduled appointment: APT002 created/updated successfully
Scheduled appointment: APT003 created/updated successfully
Scheduled appointment: APT004 created/updated successfully
Scheduled appointment: APT005 created/updated successfully
Scheduled appointment: APT006 created/updated successfully
Scheduled appointment: APT007 created/updated successfully
Scheduled appointment: APT008 created/updated successfully
Scheduled appointment: APT009 created/updated successfully
Scheduled appointment: APT010 created/updated successfully


True